# Projekt końcowy z Metod Numerycznych 2023/2024

Pytania:
- Czy można do raportu użyć markdowna? Można
- O co chodzi z upewnieniem się czy wszystkie dane są w tym samym formacie i jednostkach? 
Napisać, że z danej strony wartości są w tych jednostkach i w tych jednostkach. Nie trzeba w żaden sposób sprawdzać
- Analiza wyników???
- Analizować z całego dnia dane
- Można uśrednić z tygodnia, zrobić kwartał
- Jesli kilkuletnie to z miesiąca
- On wsm poleca kwartał
- Można z suwakiem mapkę
- Temperatura i jakies 3 inne, sumarycznie 4 wartości
- Prezentacja na 10 minut (muszą być slajdy, nie program,mapki, wykresy, podsumowania, tabele)
- Dane uzupełnić, jak coś to samemu sobie usunąć pomiary, wtedy będą rozwiązania dokładne
- mówi o aproksymacji
- wymagane są 4 stacje, powinno być z conajmniej kwartału

## Zebranie danych

Użyto danych z biblioteki meteostat. Wybrano 5 lokalizacji: 
- Kraków
- Warszawa
- Wrocław
- Szczecin
- Gdańsk

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
from meteostat import Point, Daily, units
from datetime import datetime

In [2]:
cities = {
    'Kraków': Point(50.0647, 19.9450),
    'Warszawa': Point(52.2297, 21.0122),
    'Wrocław': Point(51.1079, 17.0385),
    'Szczecin': Point(53.4285, 14.5528),
    'Gdańsk': Point(54.3520, 18.6466)
}

Dane pobrano z pierwszego kwartału 2024 roku

In [3]:
start = datetime(2024, 1, 1)
end = datetime(2024, 3, 31)

In [4]:
dataframes = []
for city, point in cities.items():
    data = Daily(point, start, end)
    data = data.fetch()
    data['City'] = city
    filename = f'{city.lower()}_weather_data.csv'
    data.to_csv(filename, index=True)
    print(f"Plik CSV dla {city} został pomyślnie wygenerowany jako {filename}.")

Plik CSV dla Kraków został pomyślnie wygenerowany jako kraków_weather_data.csv.


Plik CSV dla Warszawa został pomyślnie wygenerowany jako warszawa_weather_data.csv.


Plik CSV dla Wrocław został pomyślnie wygenerowany jako wrocław_weather_data.csv.


Plik CSV dla Szczecin został pomyślnie wygenerowany jako szczecin_weather_data.csv.


Plik CSV dla Gdańsk został pomyślnie wygenerowany jako gdańsk_weather_data.csv.


Dane zostały wygenerowane, zatem można przystąpić do obróbki danych

## Przygotowanie danych

In [9]:
df = pd.read_csv('kraków_weather_data.csv')
df.head(5)

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,City
0,2024-01-01,2.7,0.5,4.7,8.9,NaN,260.0,12.8,35.2,1009.2,NaN,Kraków
1,2024-01-02,4.9,2.9,7.0,9.9,NaN,193.0,9.7,29.6,1008.0,NaN,Kraków
2,2024-01-03,7.4,4.1,10.2,5.5,NaN,223.0,19.2,44.5,997.5,NaN,Kraków
3,2024-01-04,6.6,3.1,9.2,7.1,NaN,235.0,25.5,53.7,998.6,NaN,Kraków
4,2024-01-05,1.3,0.0,6.5,5.8,NaN,25.0,14.4,46.3,1009.8,NaN,Kraków


W tym datasecie występuje 10 zmiennych dotyczących pogody. Ograniczmy go do 4: temperatury, ciśnienia atmosferycznego, sumy opadów i prędkości wiatru

Należy usunąć pozostałe kolumny

In [11]:
df = df.drop(['tmin', 'tmax', 'snow', 'wdir', 'wpgt', 'tsun'], axis = 1)
df.head(5)

,time,tavg,prcp,wspd,pres,City
0,2024-01-01,2.7,8.9,12.8,1009.2,Kraków
1,2024-01-02,4.9,9.9,9.7,1008.0,Kraków
2,2024-01-03,7.4,5.5,19.2,997.5,Kraków
3,2024-01-04,6.6,7.1,25.5,998.6,Kraków
4,2024-01-05,1.3,5.8,14.4,1009.8,Kraków


- tavg - średnia temperatura powietrza [°C]
- wspd - średnia prędkość wiatru [km/h]
- pres - średnie ciśnienie atmosferyczne [hPa]
- prcp - średnia suma opadów [mm]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    91 non-null     object 
 1   tavg    91 non-null     float64
 2   prcp    91 non-null     float64
 3   wspd    91 non-null     float64
 4   pres    91 non-null     float64
 5   City    91 non-null     object 
dtypes: float64(4), object(2)
memory usage: 4.4+ KB


Wszystkie dane, którymi będziemy się posługiwać, są w tym samym formacie (float64)

In [13]:
df.shape

(91, 6)

Mamy do przeanalizowania łącznie 91 wierszy

In [16]:
df.isna().sum()

time    0
tavg    0
prcp    0
wspd    0
pres    0
City    0
dtype: int64

W datasecie nie występują braki, dlatego aby móc użyć metod numerycznych i sprawdzić ich efektywność, należy niektóre wartości usunąć 

In [24]:
def randomly_remove_values(df, frac, columns_to_nan):
    total_elements = df[columns_to_nan].size
    num_elements_to_remove = int(total_elements * frac)
    indices = [(row, df.columns.get_loc(col)) for row in range(df.shape[0]) for col in columns_to_nan]
    indices_to_remove = np.random.choice(range(len(indices)), num_elements_to_remove, replace=False)

    for index in indices_to_remove:
        row, col = indices[index]
        df.iat[row, col] = np.nan
    
    return df

columns_to_nan = ['tavg', 'prcp', 'wspd', 'pres']
df_with_nan = randomly_remove_values(df.copy(), 0.2, columns_to_nan)
df_with_nan.head()

,time,tavg,prcp,wspd,pres,City
0,2024-01-01,2.7,NaN,12.8,1009.2,Kraków
1,2024-01-02,4.9,9.9,NaN,1008.0,Kraków
2,2024-01-03,NaN,5.5,NaN,997.5,Kraków
3,2024-01-04,NaN,7.1,25.5,998.6,Kraków
4,2024-01-05,1.3,NaN,14.4,1009.8,Kraków


In [25]:
df_with_nan.isna().sum()

time     0
tavg    19
prcp    18
wspd    22
pres    13
City     0
dtype: int64

## Obliczanie brakujących danych

Uzupełnijmy braki za pomocą metod numerycznych. W tym przypadku najlepsza będzie aproksymacja, ponieważ nie chcemy, żeby funkcje przechodziły dokładnie przez wszystkie anomalie, ale żeby zwracały uśrednioną wartość

In [22]:
def horner_interpolation(coeffs, x):
    result = coeffs[0]
    for i in range(1, len(coeffs)):
        result = result * x + coeffs[i]
    return result

def approximate_missing_values(df, column_name, degree=3):
    indices = df.index.tolist()
    not_nan_indices = df[df[column_name].notna()].index.tolist()
    x_data = df.loc[not_nan_indices].index.tolist()
    y_data = df.loc[not_nan_indices, column_name].tolist()
    
    coeffs = np.polyfit(x_data, y_data, degree)
    
    for i in indices:
        if np.isnan(df.loc[i, column_name]):
            df.loc[i, column_name] = horner_interpolation(coeffs[::-1], i)
    
    return df

df = approximate_missing_values(df, 'snow', degree=3)

KeyError: 'snow'